<a href="https://colab.research.google.com/github/Ruhi-Sharmin-1/C-code/blob/main/XGBoost(tuned%2Bvalidated).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from scipy.io import savemat
from scipy.io import loadmat
import timeit
import numpy as np

In [24]:
from google.colab import drive
#drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [25]:
loaddir_data=F"/content/gdrive/My Drive/ml-ruhi/"

data = loadmat(loaddir_data + 'challenge_training2017_cases_normal_ecg_corr_metrics.mat',squeeze_me=True)
training_normal_features = data['all_corr_metrics'][:,:]
n_training_normal = np.shape(training_normal_features)[0]

data = loadmat(loaddir_data + 'challenge_training2017_cases_afib_ecg_corr_metrics.mat',squeeze_me=True)
training_afib_features = data['all_corr_metrics'][:,:]
n_training_afib = np.shape(training_afib_features)[0]

data = loadmat(loaddir_data + 'challenge_training2017_cases_noisy_ecg_corr_metrics.mat',squeeze_me=True)
training_noisy_features = data['all_corr_metrics'][:,:]
n_training_noisy = np.shape(training_noisy_features)[0]

data = loadmat(loaddir_data + 'challenge_training2017_cases_other_ecg_corr_metrics.mat',squeeze_me=True)
training_other_features = data['all_corr_metrics'][:,:]
n_training_other = np.shape(training_other_features)[0]

In [26]:
np.shape(training_normal_features)[0]

5050

In [27]:
# append the training datasets and learning datasets
training_features = np.concatenate((training_normal_features,training_afib_features,training_noisy_features,training_other_features),axis=0)
training_labels = np.concatenate((np.zeros(n_training_normal),np.ones(n_training_afib),2*(np.ones(n_training_noisy)),3*(np.ones(n_training_other))))

In [28]:
def nanremove(x, y):
  # input x is training_features, y is labels
  if np.argwhere(np.isnan(x)).shape[0]==0:
    return x,y
  else:
    l=np.argwhere(np.isnan(x)).shape[0]
    u=np.argwhere(np.isnan(x))
    
    for i in range(l):      
      x = np.delete(x, (u[i,0]-i), axis=0)
      y = np.delete(y, (u[i,0]-i), axis=0)
    return x,y


In [29]:
x,y=nanremove(training_features, training_labels)

In [30]:
training_all = np.concatenate((x, y.reshape((-1,1))),axis=1)
#np.random.shuffle(training_all)  #adds randomness
training_features = training_all[:,:-1]
training_labels = training_all[:,-1]

In [33]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
test_size = 0.2 # from 0.01 to 0.1
seed = 4 #change from 4 to 5
x_train, x_test, y_train, y_test = model_selection.train_test_split(training_features, training_labels, test_size=test_size, random_state=seed)

In [34]:
import xgboost as xgb
max_depth = 10
n_estimators = 110

bst = xgb.XGBClassifier(max_depth=max_depth, learning_rate=0.0001, n_estimators=n_estimators, slient=True, min_child_weight=1, objective='multi:softmax', gamma=0, reg_alpha=0, reg_lambda=1)
bst.fit(training_features, training_labels) #bst=model
# Fit the validation data  # model.predict for Y_predict
xgb_pred = bst.predict(x_test)  
# extracting most confident predictions
best_preds = np.round(xgb_pred)

In [35]:

xgb_pred_proba = bst.predict_proba(X_test)
print(Y_test)
print(xgb_pred_proba)
import pandas as pd 
pd.DataFrame(Y_test).to_csv(F"/content/gdrive/My Drive/ml-ruhi/XGBoost-Y-true-4class.csv")
pd.DataFrame(xgb_pred_proba).to_csv(F"/content/gdrive/My Drive/ml-ruhi/XGBoost-Y-pred-4class.csv")


[0. 0. 3. ... 3. 3. 3.]
[[0.25465226 0.2482424  0.24822901 0.2488763 ]
 [0.25465226 0.2482424  0.24822901 0.2488763 ]
 [0.2531725  0.24679987 0.24681361 0.253214  ]
 ...
 [0.2482368  0.24822329 0.2482099  0.25533   ]
 [0.25155297 0.24753264 0.24751928 0.2533951 ]
 [0.2507584  0.24871762 0.24868625 0.2518377 ]]


In [36]:
fimp=bst.feature_importances_

In [37]:
sorted_fimp=sorted(fimp, reverse=True)

In [38]:
sorted_fimp

[0.15094139,
 0.08984759,
 0.076137915,
 0.07167556,
 0.049601115,
 0.041528758,
 0.03515115,
 0.029978594,
 0.026282994,
 0.022806438,
 0.019364735,
 0.019212155,
 0.01714228,
 0.016465813,
 0.015936274,
 0.015799496,
 0.015327463,
 0.01526498,
 0.014746407,
 0.014688318,
 0.014179274,
 0.014102286,
 0.012907737,
 0.011867707,
 0.011632884,
 0.010560239,
 0.010460126,
 0.010408992,
 0.010332001,
 0.010218734,
 0.010009691,
 0.009921792,
 0.009679276,
 0.00956316,
 0.009319055,
 0.009179387,
 0.008845416,
 0.00847,
 0.0084580025,
 0.007918519,
 0.007689885,
 0.007551524,
 0.007513139,
 0.0068216342,
 0.0044901315,
 0.0,
 0.0,
 0.0]

In [41]:
feature_index=np.zeros(len(fimp))

In [42]:
len(fimp)

48

In [ ]:
range(len(fimp))

In [44]:
for t in range(len(fimp)):
  for h in range(len(fimp)):
    if sorted_fimp[t]==fimp[h]:
      feature_index[t]=h


In [45]:
feature_index

array([ 1.,  4.,  0., 26., 43.,  9., 31.,  7.,  6.,  3., 21., 11., 19.,
       12., 23., 10., 32., 44., 47., 14., 25., 46.,  5., 42., 18., 13.,
        8., 20., 17., 33., 27., 34., 28., 29., 22., 37., 39., 45.,  2.,
       35., 24., 30., 16., 15., 40., 41., 41., 41.])

In [47]:
feature_index_fixed = [ 0.,  1.,  4., 42., 21.,  3., 26., 11.,  9., 28.,  6., 43., 12.,
       31.,  7., 25., 23.,  5., 32., 44., 19., 29., 13., 33., 10., 27.,
       45., 17., 39.,  8., 30., 47., 35., 14., 16., 46., 34., 24., 18.,
       15., 37., 22.,  2., 20., 40., 41., 38., 36.] #fixed through observation
feature_index_fixed

[0.0,
 1.0,
 4.0,
 42.0,
 21.0,
 3.0,
 26.0,
 11.0,
 9.0,
 28.0,
 6.0,
 43.0,
 12.0,
 31.0,
 7.0,
 25.0,
 23.0,
 5.0,
 32.0,
 44.0,
 19.0,
 29.0,
 13.0,
 33.0,
 10.0,
 27.0,
 45.0,
 17.0,
 39.0,
 8.0,
 30.0,
 47.0,
 35.0,
 14.0,
 16.0,
 46.0,
 34.0,
 24.0,
 18.0,
 15.0,
 37.0,
 22.0,
 2.0,
 20.0,
 40.0,
 41.0,
 38.0,
 36.0]

In [48]:
pyplot.barh(feature_index, sorted_fimp) #did not match well maybe

NameError: ignored

In [49]:
#does not match with the for loop code and previous graphs
ax = plot_importance(bst)
ax.figure.set_size_inches(10,10)
pyplot.title('XGBoost Feature Importance Bar Chart')
pyplot.xticks(fontsize=10)
pyplot.yticks(fontsize=10)
pyplot.show()

NameError: ignored

In [50]:
from sklearn.metrics import f1_score
score = f1_score(Y_test, best_preds, average='weighted')
print('F-Measure: %.3f' % score)


F-Measure: 0.846


In [51]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test, best_preds)
accuracy * 100.0

85.12639623750735

In [52]:
best_preds.shape[0]

1701

In [53]:
best_preds_normal=[]
Y_test_normal=[]

In [54]:
for i in range(best_preds.shape[0]):
  if(Y_test[i]==0):  # if you want to do it for afib, write Y_test[i]==1
    Y_test_normal.append(Y_test[i])
    best_preds_normal.append(best_preds[i])


In [55]:
Y_test_normal=np.array(Y_test_normal)

In [56]:
best_preds_normal=np.array(best_preds_normal)

In [62]:
accuracy = accuracy_score(Y_test_normal, best_preds_normal)
accuracy * 100.0

95.83756345177665

In [64]:
from sklearn.metrics import f1_score
score = f1_score(Y_test_normal, best_preds_normal, average='weighted')
print('F-Measure: %.3f' % score)

F-Measure: 0.979


In [69]:
best_preds_afib=[]
Y_test_afib=[]

for i in range(best_preds.shape[0]):
  if(Y_test[i]==1):  # if you want to do it for afib, write Y_test[i]==1
    Y_test_afib.append(Y_test[i])
    best_preds_afib.append(best_preds[i])


Y_test_afib=np.array(Y_test_afib)
best_preds_afib=np.array(best_preds_afib)

accuracy = accuracy_score(Y_test_afib, best_preds_afib)
acc = accuracy * 100.0
print('Acc: %.3f' % acc)

from sklearn.metrics import f1_score
score = f1_score(Y_test_afib, best_preds_afib, average='weighted')
print('F-Measure: %.3f' % score)

Acc: 89.820
F-Measure: 0.946


In [71]:
best_preds_noisy=[]
Y_test_noisy=[]

for i in range(best_preds.shape[0]):
  if(Y_test[i]==2):  # if you want to do it for afib, write Y_test[i]==1
    Y_test_noisy.append(Y_test[i])
    best_preds_noisy.append(best_preds[i])


Y_test_noisy=np.array(Y_test_noisy)
best_preds_noisy=np.array(best_preds_noisy)

accuracy = accuracy_score(Y_test_noisy, best_preds_noisy)
acc = accuracy * 100.0
print('Acc: %.3f' % acc)

from sklearn.metrics import f1_score
score = f1_score(Y_test_noisy, best_preds_noisy, average='weighted')
print('F-Measure: %.3f' % score)

Acc: 82.759
F-Measure: 0.906


In [72]:
best_preds_other=[]
Y_test_other=[]

for i in range(best_preds.shape[0]):
  if(Y_test[i]==3):  # if you want to do it for afib, write Y_test[i]==1
    Y_test_other.append(Y_test[i])
    best_preds_other.append(best_preds[i])


Y_test_other=np.array(Y_test_other)
best_preds_other=np.array(best_preds_other)

accuracy = accuracy_score(Y_test_other, best_preds_other)
acc = accuracy * 100.0
print('Acc: %.3f' % acc)

from sklearn.metrics import f1_score
score = f1_score(Y_test_other, best_preds_other, average='weighted')
print('F-Measure: %.3f' % score)

Acc: 62.322
F-Measure: 0.768


In [ ]:
print('Mean ROC AUC: %.3f' % mean(score))

In [ ]:
# avg F-measure for all 3 types except noise data
from sklearn.metrics import f1_score
score = f1_score(Y_test, best_preds, average=None)
print(score) 
score = f1_score(Y_test, best_preds, average='macro')
print(score) 
score = f1_score(Y_test, best_preds, average='weighted')
print(score) 


In [ ]:
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
cm=confusion_matrix(Y_test, best_preds)

In [ ]:
# code from https://stackoverflow.com/questions/39033880/plot-confusion-matrix-sklearn-with-multiple-labels
def plot_conf_mat(cm, target_names, title='Confusion matrix', cmap=None, normalize=True):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
target_names=['Normal', 'afib', 'noisy', 'other']

In [ ]:
plot_conf_mat(cm, target_names)

In [ ]:
#cm=([[0.9715, 0, 0, 0.0286], [0.0049, 0.9323, 0.0206, 0.0422], [0.3077, 0.0769, 0.3462, 0.2692], [0.4999, 0.0556, 0.0556, 0.3889]])
#https://stackoverflow.com/questions/43691380/how-to-save-load-xgboost-model
import pickle
#file_name = "XGBoost+NN (RAW code).pkl"
file_path= F"/content/gdrive/My Drive/ml-ruhi/"

# save
pickle.dump(bst, open(file_path, "wb"))

# load
bst_loaded = pickle.load(open(file_path, "rb"))